# Building a Datapane Dashboard

We'll be working with a modified version of a database called [Chinook](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

The Chinook database is provided as a SQLite database file called `chinook.db`. It's worth remembering that our database retains 'state', so if we run a query with a `CREATE` or `DROP` twice, the query will fail. If the database gets locked, there's a `chinook-unmodified.db` file that can be copied over the `chinook.db` to restore it back to its initial state.

Here's a schema diagram for the Chinook database:

![alt text](chinook-schema.svg "Chinook Database Schema Diagram")

## Defining Helper Functions

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp

In [2]:
db = 'chinook.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c) 

In [3]:
def show_tables():
    q = '''
        SELECT
            name
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

def get_table_row_count(tablename):
    q = '''
        SELECT
            COUNT(1)
        FROM %s;
        ''' % tablename
    return run_query(q)["COUNT(1)"][0]

tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables["name"]]

tables

,name,row_count
0,album,347
1,artist,275
2,customer,59
3,employee,8
4,genre,25
5,invoice,614
6,invoice_line,4757
7,media_type,5
8,playlist,18
9,playlist_track,8715


In [4]:
import datapane as dp 

top_10_tracks_query = '''
SELECT 
    t.name trackname,
    a.title album_title,
    ar.name artist,
    count(*) as total_purchases,
    sum(il.unit_price) total_cost
FROM track t 
JOIN album a on a.album_id = t.album_id
JOIN artist ar on ar.artist_id = a.artist_id
JOIN invoice_line il on il.track_id = t.track_id
GROUP BY 1
ORDER BY total_purchases desc
'''

top_10_df = run_query(top_10_tracks_query)

## Selecting Albums to Purchase

In [5]:
albums_to_purchase = '''
WITH 
    usa_tracks_sold AS
        (
         SELECT il.* FROM invoice_line il
         INNER JOIN invoice i ON il.invoice_id = i.invoice_id
         INNER JOIN customer c ON i.customer_id = c.customer_id
        )
SELECT 
    g.name Genre, 
    COUNT(uts.invoice_line_id) "Number of Tracks Sold", 
    CAST(COUNT(uts.invoice_line_id) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold) "Percentage of Tracks Sold"
FROM usa_tracks_sold uts
INNER JOIN track t ON uts.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10;
'''

genre_sales = run_query(albums_to_purchase)
genre_sales.sort_values('Number of Tracks Sold', inplace=True)

genre_sales_plot = px.bar(
    genre_sales, 
    x="Percentage of Tracks Sold", 
    y="Genre", 
    orientation='h',
    text="Percentage of Tracks Sold",
    title = "Sales by Genre"
)

genre_sales_plot.update_traces(texttemplate='%{text:.1%}', textposition='outside')

genre_sales_plot.show()

### Observations

Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
employee_sales_performance = '''
WITH 
    customer_support_rep_sales AS
        (
         SELECT 
             i.customer_id,
             strftime('%Y-%m', i.invoice_date) || "-01" Month,
             c.support_rep_id,
             SUM(i.total) Sales_Total
         FROM invoice i
         INNER JOIN customer c ON c.customer_id = i.customer_id
         GROUP BY 2, 3
        )
SELECT
    e.first_name || " " || e.last_name "Employee Name",
    csrs.Month,
    SUM(csrs.Sales_total) "Amount of Sales (in $)"
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON csrs.support_rep_id = e.employee_id
GROUP BY 1, 2;
'''

run_query(employee_sales_performance)

,Employee Name,Month,Amount of Sales (in $)
0,Jane Peacock,2017-01-01,80.19
1,Jane Peacock,2017-02-01,82.17
2,Jane Peacock,2017-03-01,32.67
3,Jane Peacock,2017-04-01,84.15
4,Jane Peacock,2017-05-01,22.77
...,...,...,...
136,Steve Johnson,2020-08-01,25.74
137,Steve Johnson,2020-09-01,31.68
138,Steve Johnson,2020-10-01,35.64
139,Steve Johnson,2020-11-01,30.69


In [9]:
employee_sales = run_query(employee_sales_performance)
employee_sales["Month"] = pd.to_datetime(employee_sales["Month"])

employee_sales_monthly = px.line(employee_sales,
                            x = "Month",
                            y = "Amount of Sales (in $)",
                            color = "Employee Name")

employee_sales_monthly.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

employee_sales_monthly.show()

In [10]:
employee_sales_total = employee_sales.groupby(["Employee Name"]) \
                                        .sum() \
                                        .sort_values(by="Amount of Sales (in $)") \
                                        .reset_index()

employee_sales_bar = px.bar(employee_sales_total, 
                            y = 'Employee Name', 
                            x = "Amount of Sales (in $)",
                            text = "Amount of Sales (in $)",
                            orientation = 'h')

employee_sales_bar.update_traces(texttemplate='%{text:.1f}')
employee_sales_bar.show()

In [23]:
dp.Report(
    dp.Plot(employee_sales_monthly, name = "employee-sales-monthly"),
    dp.Plot(employee_sales_bar, name = "employee-sales-bar"),
).publish(name="Employee Sales", visibility=dp.Visibility.PUBLIC)

Publishing document and associated data - *please wait...*

Your report doesn't contain any text - did you know you can add text to your report from your browser once published?

InvalidTokenError: Please sign-up and login - if you already have then please restart your Jupyter kernel/Python instance to initialize your new token
Please run with `dp.enable_logging()`, restart your Jupyter kernel/Python instance, and/or visit https://www.github.com/datapane/datapane to raise issue / discuss if error repeats

### Observations

While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis.

In [14]:
sales_by_country = '''
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )


SELECT
    country,
    count(distinct customer_id) customers,
    SUM(unit_price) total_sales,
    SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
    SUM(unit_price) / count(distinct invoice_id) average_order,
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
    END AS sort
FROM country_or_other
GROUP BY country
ORDER BY sort ASC, total_sales DESC;
'''

run_query(sales_by_country)

,country,customers,total_sales,customer_lifetime_value,average_order,sort
0,USA,13,1040.49,80.037692,7.942672,0
1,Canada,8,535.59,66.948750,7.047237,0
2,Brazil,5,427.68,85.536000,7.011148,0
3,France,5,389.07,77.814000,7.781400,0
4,Germany,4,334.62,83.655000,8.161463,0
5,Czech Republic,2,273.24,136.620000,9.108000,0
6,United Kingdom,3,245.52,81.840000,8.768571,0
7,Portugal,2,185.13,92.565000,6.383793,0
8,India,2,183.15,91.575000,8.721429,0
9,Other,15,1094.94,72.996000,7.448571,1


## Visualizing Sales by Country

Now that we have our data, we've been asked to create a series of visualizations which communicate our findings, and then make recommendations on which countries may have potential for growth, so the Chinook marketing team can create some new advertising campaigns.

In [15]:
country_metrics = run_query(sales_by_country)

sales_breakdown = go.Figure(data=[go.Pie(
    labels = country_metrics["country"], 
    values=country_metrics["total_sales"], 
    hole=.3)
                                 ])

sales_breakdown.update_layout(title_text="Sales by country")

sales_breakdown.show()

In [16]:
customer_ratio = country_metrics["customers"] / country_metrics["customers"].sum() * 100.0
sales_ratio = country_metrics["total_sales"] / country_metrics["total_sales"].sum() * 100.0


custs_vs_dollars = go.Figure(data=[
    go.Bar(name='Customers', x=country_metrics["country"], y=customer_ratio),
    go.Bar(name='Total Sales', x=country_metrics["country"], y=sales_ratio),
])

# Change the bar mode
custs_vs_dollars.update_layout(barmode='group', title_text="Pct customers vs sales")
custs_vs_dollars.show()

In [17]:
country_metrics["difference_from_avg"] = (country_metrics["average_order"] - \
                                          country_metrics["average_order"].mean()) * 100.0

avg_order = px.bar(
    country_metrics, 
    x = "country", 
    y = "difference_from_avg", 
    color = "country", 
    title = "Avg order - Pct difference from mean",
    labels={
        "country": "Country",
        "difference_from_avg": "Difference from mean (%)"
    }
)

avg_order.update_layout(showlegend=False)

avg_order.show()

In [18]:
clv = px.bar(
    country_metrics,
    x = 'country',
    y = 'customer_lifetime_value',
    color='country',
    labels={
        "country": "Country",
        "customer_lifetime_value": "Customer Lifetime Value ($)"
    },
    title = "Customer Lifetime Value"
)

clv.update_layout(showlegend=False)


clv.show()

In [19]:
customers = tables[tables["name"]=="customer"]["row_count"].values[0]
employees = tables[tables["name"]=="employee"]["row_count"].values[0]
tracks_sold = tables[tables["name"]=="invoice_line"]["row_count"].values[0]

In [20]:
dp.Report(
    dp.Group(
        dp.BigNumber(heading="Total customers", value = customers),
        dp.BigNumber(heading="Total employees", value = employees),
        dp.BigNumber(heading="Total tracks sold", value = tracks_sold),
        columns = 3
    ),
    dp.Plot(genre_sales_plot, name = "genre_sales"),
    dp.Plot(employee_sales_monthly, name = "employee-sales-1"),
    dp.Plot(employee_sales_bar, name = "employee-sales-2"),
    dp.Group(
        dp.Plot(sales_breakdown),
        dp.Plot(custs_vs_dollars),
        dp.Plot(avg_order),
        dp.Plot(clv),
        columns=2
    ),
    dp.DataTable(top_10_df, name = "track-sales"),
).publish(name="Interactive Dashboard using SQLite", visibility = dp.Visibility.PUBLIC)

Publishing document and associated data - *please wait...*

Your report doesn't contain any text - did you know you can add text to your report from your browser once published?

InvalidTokenError: Please sign-up and login - if you already have then please restart your Jupyter kernel/Python instance to initialize your new token
Please run with `dp.enable_logging()`, restart your Jupyter kernel/Python instance, and/or visit https://www.github.com/datapane/datapane to raise issue / discuss if error repeats